In [2]:
import pandas as pd
import numpy as np
import psycopg2

In [88]:
conn = psycopg2.connect('dbname=football_db')

In [89]:
cur = conn.cursor()

In [90]:
columns = ['Home', 'Result', 'Count']

query = f"""
SELECT B.team_long_name,
CASE WHEN home_team_goal - away_team_goal > 0 THEN 'Win' 
      WHEN  home_team_goal - away_team_goal = 0 THEN 'Draw'
      ELSE 'Lose' END as Home_Result , count(*)
FROM Match M
JOIN Team B ON M.home_team_api_id = B.team_api_id
GROUP BY home_team_api_id, B.team_long_name, Home_Result
"""

cur.execute(query)

In [91]:
data = cur.fetchall()

In [92]:
df = pd.DataFrame(data, columns=columns)

In [94]:
df.head(10)

,Home,Result,Count
0,Le Mans FC,Lose,16
1,Eintracht Braunschweig,Win,5
2,SC Bastia,Lose,21
3,Sampdoria,Lose,35
4,Trofense,Lose,7
5,Sporting CP,Lose,18
6,Sparta Rotterdam,Lose,14
7,PEC Zwolle,Draw,17
8,Sporting Charleroi,Lose,34
9,Górnik Łęczna,Lose,10


In [63]:
df.iloc[[1, 4], :]

,Home,Away,Home_Result
1,ZUL,LOK,Draw
4,DEN,STL,Lose


In [78]:
np.random.seed(0)
samples = df.iloc[np.random.randint(low=0, high=len(df), size=100), :]

In [82]:
mu = df[df['Home_Result'] == "Win"].count() / df.count()
mu

Home           0.458717
Away           0.458717
Home_Result    0.458717
dtype: float64

In [83]:
x_bar = samples[samples['Home_Result'] == "Win"].count() / samples.count()
x_bar

Home           0.4
Away           0.4
Home_Result    0.4
dtype: float64

In [41]:
columns = ['Home_Result', 'Count']

query = f"""
SELECT Home_Result, COUNT(*) as Count FROM
(SELECT B.team_short_name as Home,  C.team_short_name as Away,  
(CASE WHEN M.home_team_goal - M.away_team_goal > 0 THEN 'Win' 
      WHEN  M.home_team_goal - M.away_team_goal = 0 THEN 'Draw'
      ELSE 'Lose' END) as Home_Result 
FROM Match M
JOIN Team B ON M.home_team_api_id = B.team_api_id
JOIN Team C ON M.away_team_api_id = C.team_api_id) AS T
GROUP BY Home_Result
"""

cur.execute(query)